In [1]:
import tensorflow as tf

# Set TensorFlow to use only one GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')  # Use GPU 0 only
        print(f"✅ TensorFlow is now using: {tf.config.list_physical_devices('GPU')}")
    except RuntimeError as e:
        print(f"❌ Runtime Error: {e}")


✅ TensorFlow is now using: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [2]:
import tensorflow as tf

# List all available GPUs
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        # Disable virtual devices by resetting logical configurations
        tf.config.experimental.set_virtual_device_configuration(
            gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024)]
        )
        
        # Do NOT set memory growth, just confirm the GPUs
        print(f"✅ TensorFlow is now using: {tf.config.list_physical_devices('GPU')}")
    
    except RuntimeError as e:
        print(f"❌ Runtime Error: {e}")

else:
    print("❌ No GPU found! TensorFlow is running on CPU.")


✅ TensorFlow is now using: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [3]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 2


In [4]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available: 2


In [5]:
import tensorflow as tf
from numba import cuda

# Reset TensorFlow sessions
tf.keras.backend.clear_session()

# Manually clear GPU memory
cuda.select_device(0)
cuda.close()

cuda.select_device(1)
cuda.close()

print("✅ Cleared GPU memory!")


✅ Cleared GPU memory!


In [6]:
import tensorflow as tf

tf.debugging.set_log_device_placement(True)
print(tf.config.list_physical_devices('GPU'))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


In [7]:
import tensorflow as tf

# List available GPUs
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    try:
        # Limit GPU memory usage
        for gpu in gpus:
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3000)]  # Adjust if needed
            )
        print("✅ GPU memory usage limited to prevent crashes.")
    except RuntimeError as e:
        print(f"❌ Error setting memory limit: {e}")
else:
    print("❌ No GPU found! TensorFlow is running on CPU.")


✅ GPU memory usage limited to prevent crashes.


In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import sys

# Memory-saving settings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Reduce TensorFlow logging
tf.config.experimental.set_memory_growth(tf.config.list_physical_devices('GPU')[0], True) if tf.config.list_physical_devices('GPU') else None

# Check available memory and alert user
def check_memory():
    try:
        import psutil
        available_memory_gb = psutil.virtual_memory().available / (1024 ** 3)
        print(f"Available memory: {available_memory_gb:.2f} GB")
        if available_memory_gb < 2:
            print("WARNING: Very low memory available. This might still fail.")
    except ImportError:
        print("psutil not installed - can't check available memory")

check_memory()

# Character-level model requires much less memory than word-level
print("Loading minimal data...")
try:
    # Only load a small subset of the data
    df = pd.read_csv("../data/cleaned_lyrics.csv", nrows=20)  # Just 20 songs
    
    # Concatenate with enough spacing
    text = "\n\n".join(df['clean_lyrics'].str[:300])  # First 300 chars of each song
    
    # Create character mapping (much smaller than word mapping)
    chars = sorted(list(set(text)))
    char_to_idx = {c: i for i, c in enumerate(chars)}
    idx_to_char = {i: c for i, c in enumerate(chars)}
    
    vocab_size = len(chars)
    print(f"Total unique characters: {vocab_size}")
    
    # Use a custom data generator to avoid loading all sequences into memory
    class DataGenerator(tf.keras.utils.Sequence):
        def __init__(self, text, char_to_idx, seq_length=40, batch_size=64):
            self.text = text
            self.char_to_idx = char_to_idx
            self.seq_length = seq_length
            self.batch_size = batch_size
            # Calculate valid starting positions (leaving room for a sequence)
            self.indices = list(range(len(text) - seq_length))
            
        def __len__(self):
            # Return number of batches
            return max(1, len(self.indices) // self.batch_size)
            
        def __getitem__(self, idx):
            # Generate one batch
            batch_indices = self.indices[idx * self.batch_size:
                                        (idx + 1) * self.batch_size]
            
            # Prepare batch data
            X = np.zeros((len(batch_indices), self.seq_length, vocab_size), dtype=np.bool_)
            y = np.zeros((len(batch_indices), vocab_size), dtype=np.bool_)
            
            for i, start_idx in enumerate(batch_indices):
                # Get sequence and target
                seq = text[start_idx:start_idx + self.seq_length]
                target = text[start_idx + self.seq_length]
                
                # One-hot encode sequence
                for t, char in enumerate(seq):
                    X[i, t, char_to_idx[char]] = 1
                
                # One-hot encode target
                y[i, char_to_idx[target]] = 1
                
            return X, y
    
    # Ultra-minimal hyperparameters
    seq_length = 40  # Sequence length
    batch_size = 32  # Small batch size
    
    # Create data generator
    data_gen = DataGenerator(text, char_to_idx, seq_length, batch_size)
    
    # Create a very tiny model
    print("Creating minimal model...")
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(seq_length, vocab_size)),
        tf.keras.layers.Dense(vocab_size, activation='softmax')
    ])
    
    # Compile with minimal settings
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
        loss='categorical_crossentropy'
    )
    
    # Train for just a few epochs
    print("Training minimal model...")
    model.fit(
        data_gen,
        epochs=5,
        verbose=1
    )
    
    # Generate a small sample
    print("\nGenerating sample text:")
    
    def generate_text(model, start_text, char_to_idx, idx_to_char, length=100):
        text = start_text
        for _ in range(length):
            # One-hot encode the last seq_length characters
            x = np.zeros((1, seq_length, vocab_size))
            for t, char in enumerate(text[-seq_length:]):
                if char in char_to_idx:
                    x[0, t, char_to_idx[char]] = 1
            
            # Predict next character
            preds = model.predict(x, verbose=0)[0]
            next_idx = np.random.choice(len(preds), p=preds)
            next_char = idx_to_char[next_idx]
            
            # Add to text
            text += next_char
            
        return text
    
    # Save a very small model
    model.save("../models/char_lyrics_generator_tiny.h5")
    print("✅ Character-level model saved successfully")
    
    # Generate small sample
    if len(text) > seq_length:
        sample = generate_text(model, text[:seq_length], char_to_idx, idx_to_char, 100)
        print(sample)
    
except Exception as e:
    print(f"Error occurred: {e}")
    print("\nIf your kernel is still crashing, you need to try these steps:")
    print("1. Restart your Jupyter kernel and run only this code")
    print("2. Try running on Google Colab instead of locally")
    print("3. Train your model using a script outside of Jupyter")
    print("4. Reduce the size of your cleaned_lyrics.csv file before loading")
    
    # If still failing, try running this minimal version
    print("\nTrying absolute minimal version as last resort...")
    
    # Create fake data if we couldn't load the real data
    if 'df' not in locals():
        text = "This is a sample lyric. I love to sing. Music is great."
        chars = sorted(list(set(text)))
        char_to_idx = {c: i for i, c in enumerate(chars)}
        idx_to_char = {i: c for i, c in enumerate(chars)}
        vocab_size = len(chars)
    
    # Ultra-minimal model that should run anywhere
    tiny_model = tf.keras.Sequential([
        tf.keras.layers.SimpleRNN(16, input_shape=(10, vocab_size)),
        tf.keras.layers.Dense(vocab_size, activation='softmax')
    ])
    
    tiny_model.compile(
        optimizer='adam',
        loss='categorical_crossentropy'
    )
    
    # Create minimal training data (just 5 examples)
    X = np.zeros((5, 10, vocab_size))
    y = np.zeros((5, vocab_size))
    
    # Just train on a single batch
    tiny_model.fit(X, y, epochs=1, verbose=1)
    
    tiny_model.save("../models/emergency_minimal_model.h5")
    print("Emergency minimal model saved - this is just a placeholder!")
    print("You'll need to train a real model in a different environment.")

Available memory: 16.79 GB
Loading minimal data...
Total unique characters: 52
Creating minimal model...
Error occurred: Memory growth cannot differ between GPU devices

If your kernel is still crashing, you need to try these steps:
1. Restart your Jupyter kernel and run only this code
2. Try running on Google Colab instead of locally
3. Train your model using a script outside of Jupyter
4. Reduce the size of your cleaned_lyrics.csv file before loading

Trying absolute minimal version as last resort...


ValueError: Memory growth cannot differ between GPU devices

In [ ]:
import matplotlib.pyplot as plt

# Plot Loss Curve
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Loss', color='red')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot Accuracy Curve (optional — accuracy isn’t too meaningful for text generation, but still useful)
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Accuracy', color='green')
plt.title('Training Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()




In [ ]:
import pickle

# Save tokenizer to file
with open("../models/tokenizer.pkl", "wb") as file:
    pickle.dump(tokenizer, file)

print("✅ Tokenizer saved to ../models/tokenizer.pkl")


In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Load the saved model
model = load_model('../models/zayn_lyrics_generator.h5')

# Load the tokenizer (make sure you saved it during training)
import pickle
with open('../models/tokenizer.pkl', 'rb') as file:
    tokenizer = pickle.load(file)

# Define a function to generate lyrics
def generate_lyrics(seed_text, next_words=50):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = np.array(token_list).reshape(1, -1)

        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=-1)[0]

        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted_word_index:
                output_word = word
                break

        seed_text += " " + output_word

    return seed_text

# Test it with a starting seed
print(generate_lyrics("I can feel", next_words=50))

